In [4]:
#######################
# LigthGBM Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import xgboost

drop_cols = ['id', 'visit_date', 'visitors', 'hpg_store_id', 'fold', 'air_store_id', 
             'air_store_id_encoded', 'hpg_store_id_encoded', 
             'air_reserved_visitors', 'hpg_reserved_visitors','reserved_visitors']

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir
MetaInputDir = '%s/meta/kfold' % DataBaseDir
OutputDir = '%s/l1/kfold' % DataBaseDir
kfold = 5
seed_num = 1
use_selected = True
strategy = 'meta_tsne'
start_time = datetime.datetime.now()
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
meta_feats = ['nn_ef', 'knn_2', 'knn_4', 'knn_8', 'knn_16', 'knn_32', 'knn_64', 'knn_128', 'knn_256', 'knn_512', 'knn_1024']
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    test = pd.read_csv('%s/test.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    for t in meta_feats:
        # load cb_ef
        FoldOutputDir = '%s/%s' % (MetaInputDir, fold)
        valid_cb_ef = pd.read_csv('%s/valid_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout_cb_ef = pd.read_csv('%s/holdout_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        test_cb_ef = pd.read_csv('%s/test_%s.csv' % (FoldOutputDir, t), parse_dates= ['visit_date']).reset_index(drop= True)
        # concate
        valid = pd.concat([valid, valid_cb_ef[[t]]], axis= 1)
        holdout = pd.concat([holdout, holdout_cb_ef[[t]]], axis= 1)
        test = pd.concat([test, test_cb_ef[[t]]], axis= 1)
        #
    valid['fold'] = fold
    valid_dfs.append(valid)
    holdout_dfs.append(holdout)
    test_dfs.append(test)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('load data done.')
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
# parameters
for fold in range(kfold):
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    ## feature options
    if(use_selected):
        selected_features = []
        with open('../../data/gfs/all/good_features_2018-01-30.txt', 'r') as i_file:
            for line in i_file:
                selected_features.append(line.rstrip())
        i_file.close()
        selected_features.extend(['nn_ef'])
#       selected_features.extend(enc_cols)
        #selected_features.extend(['air_store_id_encoded', 'hpg_store_id_encoded'])
        col = selected_features
    else:
        col = [c for c in FoldData['train'].columns if c not in drop_cols]
        col = [c for c in col if((c.startswith('count') == False) and 
                                     (c.startswith('inter') == False) and 
                                     (c.startswith('lon') == False) and 
                                     (c.startswith('lat') == False) and 
                                     (c.startswith('var') == False))]
#       col = [c for c in col if((c.startswith('count') == False) and (c.startswith('inter') == False))]
    print('feature size %s' % len(col))
    # train
    tsne = manifold.TSNE(n_components= 3, learning_rate= 500, n_iter= 500, verbose= True)
    new_cols = ['tsne_%s' % i for i in range(3)]
    FoldData['train'][new_cols] = tsne.fit_transform(FoldData['train'][col])
    FoldData['train'].drop(col, axis= 1, inplace=True)
    
    # for valid
    FoldData['valid'][new_cols] = tsne.transform(FoldData['valid'][col])
    FoldData['valid'].drop(col, axis= 1, inplace=True)

    # for holdout
    FoldData['holdout'][new_cols] = tsne.transform(FoldData['holdout'][col])
    FoldData['holdout'].drop(col, axis= 1, inplace=True)

    # for test
    FoldData['test'][new_cols] = tsne.transform(FoldData['test'][col])
    FoldData['test'].drop(col, axis= 1, inplace=True)

    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    print('saving for %sth fold data done.' % (fold))
    
# y_test_pred /= kfold  # Average test set predictions
# cv_score /= kfold # Average valid set predictions
# holdout_score /= kfold # Average holdout set predictions

# # Create submission file
# sub = pd.DataFrame()
# sub['id'] = test_dfs[0]['id']
# sub['visitors'] = np.expm1(y_test_pred)
# OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
# SubmitDir = '%s/l1/submit' % DataBaseDir
# if(os.path.exists(SubmitDir) == False):
#     os.makedirs(SubmitDir)
# sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
# os.system('zip %s/%s.zip %s.csv' % (SubmitDir, OutputFileName, OutputFileName))

# finish_time = datetime.datetime.now()
# elapsed = (finish_time - start_time).seconds
# print('\n======================')
# print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
# print('======================\n')

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


load data done.
feature size 51
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 181482 samples in 0.742s...
[t-SNE] Computed neighbors for 181482 samples in 150.816s...
[t-SNE] Computed conditional probabilities for sample 1000 / 181482
[t-SNE] Computed conditional probabilities for sample 2000 / 181482
[t-SNE] Computed conditional probabilities for sample 3000 / 181482
[t-SNE] Computed conditional probabilities for sample 4000 / 181482
[t-SNE] Computed conditional probabilities for sample 5000 / 181482
[t-SNE] Computed conditional probabilities for sample 6000 / 181482
[t-SNE] Computed conditional probabilities for sample 7000 / 181482
[t-SNE] Computed conditional probabilities for sample 8000 / 181482
[t-SNE] Computed conditional probabilities for sample 9000 / 181482
[t-SNE] Computed conditional probabilities for sample 10000 / 181482
[t-SNE] Computed conditional probabilities for sample 11000 / 181482
[t-SNE] Computed conditional probabilities for sample 12000 / 181482
[t

[t-SNE] Computed conditional probabilities for sample 121000 / 181482
[t-SNE] Computed conditional probabilities for sample 122000 / 181482
[t-SNE] Computed conditional probabilities for sample 123000 / 181482
[t-SNE] Computed conditional probabilities for sample 124000 / 181482
[t-SNE] Computed conditional probabilities for sample 125000 / 181482
[t-SNE] Computed conditional probabilities for sample 126000 / 181482
[t-SNE] Computed conditional probabilities for sample 127000 / 181482
[t-SNE] Computed conditional probabilities for sample 128000 / 181482
[t-SNE] Computed conditional probabilities for sample 129000 / 181482
[t-SNE] Computed conditional probabilities for sample 130000 / 181482
[t-SNE] Computed conditional probabilities for sample 131000 / 181482
[t-SNE] Computed conditional probabilities for sample 132000 / 181482
[t-SNE] Computed conditional probabilities for sample 133000 / 181482
[t-SNE] Computed conditional probabilities for sample 134000 / 181482
[t-SNE] Computed con

KeyboardInterrupt: 